# Incremental Ernollment Trial with voxceleb1
---------

trial 길이가 충분히 긴 set을 만들고자 한다.

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pickle
import pandas as pd

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append("../")

In [3]:
from batch_sv_system import get_embeds, cosine_sim, compute_plda_score
from utils import key2df, df2dict, get_id2idx
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support
from tqdm  import tqdm

## Dataframes & embeddings

In [4]:
# voxc1_df['label'] = voxc1_df.groupby('spk').ngroup().values
# voxc1_df = voxc1_df.sort_values('id').reset_index(drop=True)
# voxc1_df = voxc1_df.fillna('test')
# voxc1_meta = pd.read_pickle("/dataset/SV_sets/voxceleb1/dataframes/voxc1_meta.pkl")
# spk2gender = voxc1_meta.Gender.to_dict()
# voxc1_df['gender'] = voxc1_df.spk.apply(lambda x: spk2gender[x])
# voxc1_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1.csv", index=False)

# voxc1_dev_df = voxc1_df[voxc1_df.spk.isin(dev_spks)]
# voxc1_eval_df = voxc1_df[voxc1_df.spk.isin(eval_spks)]
# voxc1_dev_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_dev.csv", index=False)
# voxc1_eval_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_eval.csv", index=False)

In [5]:
voxc1_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1.csv")
spk_uttr_stat = voxc1_df.spk.value_counts()
voxc1_meta = pd.read_pickle("/dataset/SV_sets/voxceleb1/dataframes/voxc1_meta.pkl")
spk2gender = voxc1_meta.Gender.to_dict()

In [6]:
from utils import key2df

embed_dir = "../embeddings/voxc2_fbank64_voxc2untied_xvector/"
sv_embeds = np.load(embed_dir+"ln_lda_sv_embeds.npy")
sv_keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)

In [7]:
from batch_sv_system import compute_plda_score
kaldi_plda_model = embed_dir+"plda_train/"

In [8]:
from ioffe_plda.verifier import Verifier
py_plda_model = Verifier()
py_plda_model = Verifier(pickle.load(open("../py_plda_model_ln_lda.pkl", "rb")))

## Hold-out Developement set

utterance가 150개 이상인 화자들을 evaluation spks로 빼고 나머지는 developement spk로 빼었다.  
그리고 developement spk 음성을 이용해서 threshold를 정하기 위한 trial을 만든다.

In [26]:
dev_spks = spk_uttr_stat[(spk_uttr_stat < 150)].index.tolist()
dev_uttrs = voxc1_df[voxc1_df.spk.isin(dev_spks)][['id', 'spk', 'gender', 'session']]
eval_spks = spk_uttr_stat[spk_uttr_stat >= 150].index.tolist()
eval_uttrs = voxc1_df[voxc1_df.spk.isin(eval_spks)][['id', 'spk', 'gender', 'session']]

In [10]:
print(f"number of dev spks: {len(dev_spks)}")
print(f"number of eval spks: {len(eval_spks)}")

number of dev spks: 940
number of eval spks: 311


## Cohort Set

In [62]:
cohort_ids = dev_uttrs.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=20)).id
np.save("trials/dev940_eval311/dev_cohort_ids.npy", np.array(cohort_ids))

## Dev trials for eT 

### Original trials 

In [11]:
voxc2_trial = pd.read_pickle("../trials/voxc2_1211_trials.pkl")
trial_for_thresh = voxc2_trial[(voxc2_trial.enroll_spk.isin(dev_spks)) 
                               & (voxc2_trial.test_spk.isin(dev_spks))]

dev_trials = trial_for_thresh[["enroll_idx", "test_idx", "label"]]
dev_trials.columns = ["enr_id", "test_id", "label"]
dev_trials = dev_trials.reset_index(drop=True)

In [12]:
dev_trials.to_pickle("../trials/dev940_eval311/dev_original_trials.pkl")

In [13]:
dev_trials.label.value_counts()

1    157309
0     85172
Name: label, dtype: int64

### Dev Target trials 

In [55]:
import itertools

dev_combs = dev_uttrs.groupby("spk", group_keys=False).apply(lambda x: itertools.combinations(x.id, 2))
dev_target_trials = []
for comb in dev_combs:
    dev_target_trials += list(comb)

dev_target_trial_df = pd.DataFrame(dev_target_trials, columns=['enr', 'tst'])
dev_target_trial_set = dev_target_trial_df.groupby('enr', group_keys=False).apply(
    lambda x: list(x.tst)).to_dict()
# pickle.dump(dev_target_trial_set, open("trials/dev940_eval311/dev_target_trials.pkl", "wb"))

In [56]:
len(dev_target_trial_set)

82141

## Evaluation trials

In [28]:
n_target_adapt = 150
adapt_uttrs = eval_uttrs.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=n_target_adapt))
test_uttrs = eval_uttrs.drop(index=adapt_uttrs.index)

### Inter speaker scores 

In [17]:
spk_embeds = []
for spk in eval_spks:
    embeds = get_embeds(eval_uttrs[eval_uttrs.spk == spk].id, sv_embeds, sv_id2idx, norm=False).mean(0)
    spk_embeds.append(embeds)

spk_embeds = np.stack(spk_embeds, axis=0)

In [18]:
inter_spk_scores = py_plda_model.score_avg(spk_embeds, spk_embeds)

### Spaker similarity sorting 

In [19]:
sorted_groups = np.argsort(inter_spk_scores, axis=1)[:, :-1]

In [20]:
spk_idx = 0
close_spk_idx = sorted_groups[spk_idx][-10:]
close_spks = np.array(eval_spks)[close_spk_idx]
inter_spk_scores[spk_idx][close_spk_idx]
hard_neg_embeds = get_embeds(eval_uttrs[eval_uttrs.spk.isin(close_spks)].id, sv_embeds, sv_id2idx, norm=False)
hard_neg_scores = py_plda_model.multi_sess(spk_embeds[spk_idx], hard_neg_embeds)

In [21]:
hard_neg_scores[-1000:]

array([[-9.70471, -2.00652, -1.07242, ..., -7.32827, -1.32971, -4.01678]])

### Speaker clustering 

In [22]:
from sklearn.cluster import AgglomerativeClustering

cluster_model = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')
cluster_ids = cluster_model.fit_predict(inter_spk_scores)

In [23]:
np.unique(cluster_ids, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([54, 29, 50, 50, 38, 25, 18, 12, 28,  7]))

In [54]:
n_enr = 3 
n_repeat_enr = 5
l_trial = 300
p_tar = 0.1

adapt_trials = []

for spk_idx, adapt_spk in tqdm(enumerate(eval_spks), total=len(eval_spks)):
    # target and nontarget utterances
    target_uttrs = adapt_uttrs[adapt_uttrs.spk == adapt_spk]
    target_uttrs.loc[:, 'label'] = 1
    
    # cluster based nontarget pool
    own_id = cluster_ids[spk_idx]
    nonTarget_spks = np.array(eval_spks)[cluster_ids==own_id]
    nonTarget_spks = set(nonTarget_spks) - set([adapt_spk]) 
    nonTarget_adapt_uttrs = adapt_uttrs[adapt_uttrs.spk.isin(nonTarget_spks)]
    nonTarget_adapt_uttrs.loc[:, 'label'] = 0
    # enrollment set
    enr_uttrs_set = target_uttrs.sample(n=n_enr*n_repeat_enr, replace=True)
    target_adapt_uttrs = target_uttrs.drop(index=enr_uttrs_set.index)
    
    for idx in range(0, len(enr_uttrs_set), n_enr):
        enr_uttrs = enr_uttrs_set[idx:idx+n_enr]
        
        # number of targets and nontargets
        n_adapt_target_uttrs = int(l_trial * p_tar)
        target_adapt_uttrs_ = target_adapt_uttrs.sample(n=n_adapt_target_uttrs)
        n_nonTargets = l_trial - n_adapt_target_uttrs
        
        nonTarget_idx = np.random.choice(np.arange(len(nonTarget_adapt_uttrs)), size=n_nonTargets, replace=False)
        nonTarget_adapt_uttrs_ = nonTarget_adapt_uttrs.iloc[nonTarget_idx]

        adapt_trial = pd.concat([target_adapt_uttrs_, nonTarget_adapt_uttrs_])
        adapt_trial = adapt_trial.sample(frac=1.0)
        adapt_trials += [({'spk':adapt_spk, "p_tar":p_tar}, np.array(enr_uttrs.id), adapt_trial)]

  0%|          | 0/311 [00:00<?, ?it/s]/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
100%|██████████| 311/311 [01:01<00:00,  5.09it/s]


In [55]:
pickle.dump(adapt_trials, open(f"../trials/dev940_eval311/split_trials/adapt_len{l_trial}_enr{n_enr}_ptar{p_tar}_cluster_trials.pkl", "wb"))

### Adapt+Test trials

이전처럼 adapt와 test trial을 같이 생성한다.
그래야 실제 환경에 더 맞는다.

In [278]:
n_enr = 1 
n_repeat_enr = 1
l_trial = 1000
p_tar = 0.1
p_hard = 0.9

adapt_trials = []

for adapt_spk in tqdm(eval_spks, total=len(eval_spks)):
    target_uttrs = target_adapt_trials[target_adapt_trials.spk == adapt_spk]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_adapt_uttrs = target_adapt_trials[target_adapt_trials.spk != adapt_spk]
    nonTarget_adapt_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs_set = target_uttrs.sample(n=n_enr*n_repeat_enr, replace=True)
    target_adapt_uttrs = target_uttrs.drop(index=enr_uttrs_set.index)
    for idx in range(0, len(enr_uttrs_set), n_enr):
        enr_uttrs = enr_uttrs_set[idx:idx+n_enr]
        
        ### Adapt target and nonTarget
        n_adapt_target_uttrs = int(l_trial * p_tar)
        target_adapt_uttrs_ = target_adapt_uttrs.sample(n=n_adapt_target_uttrs)
        
        n_nonTargets = l_trial - n_adapt_target_uttrs
        n_hard_nonTargets = np.ceil(n_nonTargets * p_hard).astype(np.int64)
        n_random_nonTargets = n_nonTargets - n_hard_nonTargets
        
        if n_hard_nonTargets > 0:
            # init_enroll dependent hard trials
            enr_embeds = get_embeds(enr_uttrs.id, sv_embeds, sv_id2idx, norm=False)
            nonTarget_test_embeds = get_embeds(nonTarget_adapt_uttrs.id, sv_embeds, sv_id2idx, norm=False)
            nonTarget_scores = py_plda_model.multi_sess(enr_embeds, nontarget_test_embeds, cov_scaling=True).mean(0)
            nonTarget_sorted_idx = np.argsort(nonTarget_scores)
            nonTarget_hard_idx = nonTarget_sorted_idx[-n_hard_nonTargets:]
            nonTarget_random_idx = np.random.choice(nonTarget_sorted_idx[:-n_hard_nonTargets], size=n_random_nonTargets, replace=False)
            nonTarget_idx = np.concatenate([nonTarget_hard_idx, nonTarget_random_idx])
            
            # spk depentdent hard trials
        else:
            nonTarget_idx = np.random.choice(np.arange(len(nonTarget_adapt_uttrs)), size=n_random_nonTargets, replace=False)

        nonTarget_adapt_uttrs_ = nonTarget_adapt_uttrs.iloc[nonTarget_idx]

        adapt_trial = pd.concat([target_adapt_uttrs_, nonTarget_adapt_uttrs_])
        adapt_trial = adapt_trial.sample(frac=1.0)
        adapt_trials += [({'spk':adapt_spk, "p_tar":p_tar, "p_hard":p_hard}, np.array(enr_uttrs.id), adapt_trial)]

  0%|          | 0/311 [00:00<?, ?it/s]/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s



ValueError: Cannot take a larger sample than population when 'replace=False'

In [139]:
pickle.dump(adapt_trials, open(f"../trials/dev940_eval311/split_trials/adapt_len{l_trial}_enr{n_enr}_hard{p_hard}_ptar{p_tar}_trials.pkl", "wb"))

### Test trials 

In [110]:
test_trials = {}

for test_spk in tqdm(eval_spks, total=len(eval_spks)):
    target_test_uttrs = target_test_trials[target_test_trials.spk == test_spk]
    nonTarget_test_uttrs = target_test_trials[target_test_trials.spk != test_spk]
    target_test_uttrs.loc[:, 'label'] = 1
    nonTarget_test_uttrs.loc[:, 'label'] = 0
    
    ## test trial 
#     target_test_embeds = get_embeds(target_test_uttrs.id, sv_embeds, sv_id2idx, norm=False)
#     nontarget_test_embeds = get_embeds(nontarget_test_uttrs.id, sv_embeds, sv_id2idx, norm=False)
#     nontarget_scores = py_plda_model.multi_sess(target_test_embeds, nontarget_test_embeds, cov_scaling=true).mean(0)
#     # use only half of nonTarget
#     nonTarget_sorted_idx = np.argsort(nonTarget_scores)[-len(nonTarget_scores)//2:]
# #     nonTarget_idx = nonTarget_sorted_idx[-len(target_test_uttrs):]
#     nonTarget_idx = np.random.choice(nonTarget_sorted_idx, size=len(target_test_uttrs), replace=False)
#     nonTarget_test_uttrs = nonTarget_test_uttrs.iloc[nonTarget_idx]

    nonTarget_test_uttrs = nonTarget_test_uttrs.sample(n=2000)
    
#     print(target_test_uttrs.shape)
#     print(nonTarget_test_uttrs.shape)

    test_trial = pd.concat([target_test_uttrs, nonTarget_test_uttrs])
    test_trial = test_trial.sample(frac=1.0)
    test_trials[test_spk] = test_trial


100%|██████████| 311/311 [01:00<00:00,  5.16it/s]


In [112]:
pickle.dump(test_trials, open("../trials/dev940_eval311/split_trials/test_random_trials.pkl", "wb"))

In [213]:
enr_embeds = get_embeds(adapt_trials[0][1], sv_embeds, sv_id2idx, norm=False)
nonT_embeds = get_embeds(nonTarget_test_uttrs.id, sv_embeds, sv_id2idx, norm=False)

In [216]:
# nonTarget_scores = py_plda_model.multi_sess(enr_embeds, nonT_embeds, cov_scaling=True).mean(0)
nonTarget_scores = py_plda_model.multi_sess(target_test_embeds, nonT_embeds, cov_scaling=True).mean(0)

## Closed Evaluation Set 

In [74]:
spk_embed_dict = eval_uttrs.groupby('spk', group_keys=False).apply(lambda x: get_embeds(x.id, sv_embeds, sv_id2idx, norm=False).mean(0))